<a href="https://colab.research.google.com/github/ranim2001-lab/AIPersonalizedLearningPlatform-main/blob/main/meeting_summarizer_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install transformers torch

In [4]:
!pip install transformers accelerate torch soundfile ffmpeg-python


In [5]:
!pip install bitsandbytes sentencepiece


In [6]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
import subprocess

device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

In [9]:
import os

print(os.listdir())


['.config', '.ipynb_checkpoints', 'meeting.mp4', 'meeting.mp3', 'transcription.txt', 'sample_data']


In [10]:
import subprocess

def convert_mp4_to_mp3(input_file, output_file):
    command = f"ffmpeg -y -i {input_file} -ac 1 -ar 16000 {output_file}"
    result = subprocess.run(command.split(), capture_output=True)

    if result.returncode == 0:
        print("Conversion OK :", output_file)
    else:
        print("Erreur FFmpeg :")
        print(result.stderr.decode())


In [11]:
convert_mp4_to_mp3("meeting.mp4", "meeting.mp3")


Conversion OK : meeting.mp3


In [16]:
from transformers import pipeline

pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-small",
    chunk_length_s=30,
    stride_length_s=5,
    return_timestamps=True,
    device=device,
    generate_kwargs={"language": "english", "task": "transcribe"}
)

transcription = pipe("meeting.mp3")


Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


In [17]:
formatted_lyrics = ""
for line in transcription['chunks']:
    text = line["text"]
    ts = line["timestamp"]
    formatted_lyrics += f"{ts}-->{text}\n"

print(formatted_lyrics.strip())


(0.0, 14.84)--> No, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no,
(14.84, 20.88)--> no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no,
(20.88, 25.92)--> no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no,
(25.92, 29.84)--> no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no,
(29.84, 31.84)--> Thank you very much for watching.


In [18]:
with open("transcription.txt", "w", encoding="utf-8") as file:
    file.write(formatted_lyrics.strip())

print("Transcription saved to transcription.txt")


Transcription saved to transcription.txt


**SUMMARIZING WITH LLAMA**

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 🔥 Modèle GRATUIT et non-gated
DEFAULT_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"

# Chargement du modèle
model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL)
tokenizer.pad_token_id = tokenizer.eos_token_id

print("✅ Qwen2.5-1.5B chargé avec succès !")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ Qwen2.5-1.5B chargé avec succès !


In [19]:
conversation = [
    {"role": "system", "content": "Summarize the meeting transcript clearly and concisely."},
    {"role": "user", "content": formatted_lyrics.strip()},
]

# Préparer le prompt au format Qwen
prompt = tokenizer.apply_chat_template(conversation, tokenize=False)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=500,
        do_sample=True,
        temperature=0.4,
        top_p=0.9,
    )

summary = tokenizer.decode(output[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
print("📌 SUMMARY:\n")
print(summary)


📌 SUMMARY:

system
The meeting ended with a thank-you message from one participant to another.


In [22]:
import os
os.listdir()


['.config',
 '.ipynb_checkpoints',
 'meeting.mp4',
 'meeting.mp3',
 'transcription.txt',
 'sample_data']

In [25]:
import json

# Ton fichier actuel
input_nb = "meeting_summarizer_project.ipynb"
output_nb = "meeting_summarizer_project_clean.ipynb"

with open(input_nb, "r", encoding="utf-8") as f:
    nb = json.load(f)

# Supprimer metadata.widgets si existe
if "widgets" in nb.get("metadata", {}):
    del nb["metadata"]["widgets"]

# Supprimer 'state' dans toutes les cellules si trouvé
for cell in nb["cells"]:
    if "metadata" in cell and "widgets" in cell["metadata"]:
        if "state" in cell["metadata"]["widgets"]:
            del cell["metadata"]["widgets"]["state"]

with open(output_nb, "w", encoding="utf-8") as f:
    json.dump(nb, f, indent=2)

print("Notebook cleaned ➜", output_nb)


FileNotFoundError: [Errno 2] No such file or directory: 'meeting_summarizer_project.ipynb'

In [26]:
import os
os.listdir()


['.config',
 '.ipynb_checkpoints',
 'meeting.mp4',
 'meeting.mp3',
 'transcription.txt',
 'sample_data']

In [27]:
from google.colab import files

files.download("/content/meeting_summarizer_project.ipynb")



FileNotFoundError: Cannot find file: /content/meeting_summarizer_project.ipynb

In [28]:
from google.colab import files
uploaded = files.upload()


Saving meeting_summarizer_project.ipynb to meeting_summarizer_project.ipynb


In [29]:
!ls -l


total 19024
-rw-r--r-- 1 root root    45091 Nov 14 23:05 meeting.mp3
-rw-r--r-- 1 root root 19322418 Nov 14 21:31 meeting.mp4
-rw-r--r-- 1 root root    95643 Nov 14 23:49 meeting_summarizer_project.ipynb
drwxr-xr-x 1 root root     4096 Nov 12 14:30 sample_data
-rw-r--r-- 1 root root      473 Nov 14 23:13 transcription.txt


In [30]:
from google.colab import files
files.download("meeting_summarizer_project.ipynb")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
import json

input_nb = "meeting_summarizer_project.ipynb"
output_nb = "meeting_summarizer_project_clean.ipynb"

with open(input_nb, "r", encoding="utf-8") as f:
    nb = json.load(f)

# supprimer toutes les cellules de widgets
for cell in nb["cells"]:
    if "metadata" in cell and "widgets" in cell["metadata"]:
        del cell["metadata"]["widgets"]

with open(output_nb, "w", encoding="utf-8") as f:
    json.dump(nb, f, indent=2)

print("Clean notebook saved as", output_nb)


Clean notebook saved as meeting_summarizer_project_clean.ipynb


In [32]:
files.download("meeting_summarizer_project_clean.ipynb")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>